In [1]:
import nltk
from nltk.tokenize import word_tokenize
import re
import numpy
import fasttext

In [2]:
def tokenization(query):
    tokens=query.upper()
    if "WHERE" in tokens:
        tokens=tokens.split("WHERE",1)
        tokens=tokens[1]
    Dict = { '!=': 'NEQ', '<>': 'NEQ', '&&': 'AND', '||': 'OR', '/*': ' CMTST ',
            '*/': 'CMTEND',
            '~': 'TLDE', '!': 'EXCLM', '@': 'ATR', '#': 'HASH', '$': 'DLLR', '%': 'PRCNT',
            '^': 'XOR', '&': 'BITAND','|': 'BITOR', '*': 'STAR', '--': 'CMNT', '-': 'MINUS',
            '+': 'PLUS', '=': 'EQ', '/': 'SLSH', '?': 'QSTN','.': 'DOT',
            ',': 'CMMA', '>': 'GT', '<': 'LT', '\'': 'SQUT', '"': 'DQUT', ';': 'SMCLN', 
            ':': 'CLN', '\\': 'BSLSH',
            ']': 'RSQBR', '[': 'LSQBR', '}': 'RCBR', '{': 'LCBR', ')': 'RPRN', '(': 'LPRN'}
    i=0 #counter
    while i < len(tokens):
        if tokens[i] in Dict: #if token is in dict
            if i + 1 < len(tokens) and (tokens[i] + tokens[i + 1]) in Dict: #if two things after each other replace such as ||
                tokens=tokens.replace((tokens[i] + tokens[i + 1])," "+Dict[(tokens[i] + tokens[i + 1])]+" ")
            else:
                tokens=tokens.replace(tokens[i], " "+Dict[tokens[i]]+" ")# if only 1 replace it
           
        i = i + 1 #increment counter
  
    tokens=word_tokenize(tokens) #nltk function to split string into tokens
    i=0
    while i < len(tokens):
        if re.search("^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$", tokens[i]):
            tokens[i] = "IPADDR"
        elif re.search("^([0-9]+)$", tokens[i]):
            tokens[i] = "INT"
        elif re.search("^([A-Fa-f0-9]{2,})$", tokens[i]):
            tokens[i] = "HEX"
        elif re.search("^\d*[.,]?\d*$", tokens[i]):
            tokens[i] = "DEC"
        elif re.search("^([A-Za-z])$", tokens[i]):
            tokens[i] = "CHR"
        else:
            if i-1 > 0 and tokens[i-1] == "FROM":
                tokens[i]= "TABLE"
        i = i + 1 #increment counter        
    return " ".join(tokens) # convert it to string again

In [3]:
queries = open("/home/mariam/Desktop/dataset/queries.txt", "rt")
tokenized = open("/home/mariam/Desktop/dataset/tokenized.txt", "w")

In [4]:
counter=0
while True:
    counter=counter+1
    line=queries.readline() #take the query from queries file
    if not line: #if there is no query stop
        break
    line.strip() 
    line=line.split(" ",1) # to split the prefix label
    query=tokenization(line[1]) # call tokenization function
    tokenized.writelines([line[0]+" "+query,"\n"]) # write tokenized string in the file called "tokenized"
queries.close()
tokenized.close() 

In [5]:
training=open("/home/mariam/Desktop/dataset/train.txt", "w") #450 malicious #407 non-malicious #training
test=open("/home/mariam/Desktop/dataset/test.txt", "w") # 150 malicious #100 non-malicios #testing
tokenized = open("/home/mariam/Desktop/dataset/tokenized.txt", "r")


In [6]:
counter=0
while True:
    counter=counter+1
    line=tokenized.readline() #read tokenized query
    if not line:
        break
    line.strip()
    if counter<=407: #fill 407 non malicious in training
        training.writelines([line,"\n"])
    elif counter>407 and counter<=507: #fill 100 non malicious in valid
        test.writelines([line,"\n"])
    if counter>507 and counter<=957: #fill 450 malicious in training
        training.writelines([line,"\n"])
    elif counter>957 and counter<=1107: #fill 150 non malicious in valid
        test.writelines([line,"\n"])

        
        
test.close()    
tokenized.close()
training.close()

In [7]:
model = fasttext.train_supervised('/home/mariam/Desktop/dataset/train.txt') #train the model using training file

In [8]:
#print(model.test('/home/mariam/Desktop/dataset/valid.txt'))
total, precision, recall = model.test('/home/mariam/Desktop/dataset/test.txt') #test the model
Fmeasure= 2 * ((precision*recall)/(precision + recall)) #calc f-measure
print(str(Fmeasure * 100) + "%" )

94.0%


In [9]:

texts = ['CUSTOMERNAME LIKE SQUT _R PRCNT SQUT SMCLN',
        'UPDATE USERS SET EMAIL EQ LOWER LPRN EMAIL RPRN SMCLN',
        'DROP TABLE IF EXISTS CUSTOMER CASCADE SMCLN',
        'COUNTRY IN LPRN SQUT USA SQUT CMMA SQUT UK SQUT CMMA SQUT JAPAN SQUT RPRN',
        'INSERT INTO CUSTOMER LPRN NAME RPRN SELECT RANDOM LPRN RPRN CLN CLN TEXT FROM TABLE LPRN INT CMMA INT RPRN SMCLN',
        'INT PRCNT DQUT RPRN RPRN RPRN UNION ALL SELECT NULL CMMA NULL CMMA NULL CMMA NULL CMMA NULL CMMA NULL CMMA NULL CMMA NULL CMNT',
        'INT EQ INT AND LPRN SELECT STAR FROM TABLE SELECT LPRN SLEEP LPRN INT RPRN RPRN RPRN FZNO RPRN CMNT',
        'ID EQ INT OR LPRN BSLSH DOT RPRN EQ INT UNION SELECT INT CMMA ATR ATR VERSION CMNT INT',
        'DECLARE ATR CHR NVARCHAR LPRN INT RPRN 0X730065006C00650063007400200040004000760065007200730069006F006E00 EXEC LPRN ATR CHR RPRN',
        'OR CHR EQ CHR']
    
labels = model.predict(texts)
for i in labels[0]:
    print (i , "\n")
        

['__label__nonmalicious'] 

['__label__nonmalicious'] 

['__label__nonmalicious'] 

['__label__nonmalicious'] 

['__label__nonmalicious'] 

['__label__malicious'] 

['__label__malicious'] 

['__label__malicious'] 

['__label__malicious'] 

['__label__malicious'] 



In [12]:
print(tokenization('select * from users where id = 1 +$ . or 1 = 1 -- 1'))

ID EQ INT PLUS DLLR DOT OR INT EQ INT CMNT INT


In [13]:
model.predict('ID EQ INT PLUS DLLR DOT OR INT EQ INT CMNT INT')

(('__label__malicious',), array([0.98918331]))

In [ ]:
print("THANK YOU")